In [16]:
import nltk
import pandas as pd
import ast
import re
import numpy as np

import string 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')

import unidecode
import nltk.corpus
from gensim.models import Word2Vec

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import config
import torch
from transformers import BertTokenizer, BertModel

[nltk_data] Error loading wordnet: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>


In [17]:
df = pd.read_csv("/Users/ved/Desktop/Capstone/Dataset/Cleaned_Indian_Food_Dataset.csv")

In [18]:
df.rename(columns = {'Cleaned-Ingredients':'ingredients'}, inplace = True)
for i in range(len(df["ingredients"])):
    df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
    df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
    df["ingredients"][i] = df["ingredients"][i].split(',')

/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_38337/1364071601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_38337/1364071601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_38337/1364071601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [19]:
df.to_csv('/Users/ved/Desktop/cap/dataset.csv')

In [20]:
print(df["ingredients"][0])

['salt', 'amchur ', 'karela ', 'red chilli powder', 'gram flour ', 'onion', 'cumin seeds ', 'coriander powder', 'turmeric powder', 'sunflower oil']


In [21]:
def ingredient_parser(ingredients):
    # measures and common words (already lemmatized)   
    measures = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', ...]
    words_to_remove = ['fresh', 'a', 'red', 'bunch', ...]
    # Turn ingredient list from string into a list 
    if isinstance(ingredients, list):
       pass
    else:
       ingredients = ast.literal_eval(ingredients)
    # We first get rid of all the punctuation
    remove_punctuations = str.maketrans('', '', string.punctuation)
    # initialize nltk's lemmatizer    
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for each_item in ingredients:
        each_item.translate(remove_punctuations)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', each_item)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        items = [unidecode.unidecode(word) for word in items]
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # get rid of stop words
        stop_words = set(nltk.corpus.stopwords.words('english'))
        items = [word for word in items if word not in stop_words]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        items = [word for word in items if word not in measures]
        # Get rid of common easy words
        items = [word for word in items if word not in words_to_remove]
        if items:
           ingred_list.append(' '.join(items))
    return ingred_list

In [22]:
df['parsed'] = df.ingredients.apply(ingredient_parser)

In [23]:
def ingredient_parser_final(ingredient):
    """
    neaten the ingredients being outputted
    """
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        ingredients = ast.literal_eval(ingredient)

    ingredients = ",".join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients
def title_parser(title):
    title = unidecode.unidecode(title)
    return title

In [24]:
a = []

In [25]:
a = pd.DataFrame(a)

In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GlobalAveragePooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




def process_ingredients(df, column_name):
    # Extract the ingredient column from the DataFrame
    ingredient_lists = df[column_name].tolist()
    
    # Tokenize and create word embeddings
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(ingredient_lists)
    vocab_size = len(tokenizer.word_index) + 1  # +1 for OOV token

    # Convert ingredients to sequences
    sequences = tokenizer.texts_to_sequences(ingredient_lists)

    # Pad sequences for consistent input size
    max_sequence_length = max(len(seq) for seq in sequences)
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

    # Create a model for word embeddings
    word_embedding_model = Sequential()
    word_embedding_model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_length))
    word_embedding_model.compile(loss='categorical_crossentropy', optimizer='adam')
    word_embeddings = word_embedding_model.predict(padded_sequences)

    # Create a model for document embeddings
    document_embedding_model = Sequential()
    document_embedding_model.add(LSTM(100))
    document_embedding_model.compile(loss='categorical_crossentropy', optimizer='adam')
    document_embeddings = document_embedding_model.predict(word_embeddings)

    # Add the embeddings to the DataFrame
    df['word_embeddings'] = list(word_embeddings)
    df['document_embeddings'] = list(document_embeddings)

# Process the ingredients column in the DataFrame
process_ingredients(df, 'parsed')

# Display the updated DataFrame


def process_ingredients_input(abc):
    # Extract the ingredient column from the DataFrame
    
    # Tokenize and create word embeddings
    ingredient_lists = abc
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(ingredient_lists)
    vocab_size = len(tokenizer.word_index) + 1  # +1 for OOV token

    # Convert ingredients to sequences
    sequences = tokenizer.texts_to_sequences(ingredient_lists)

    # Pad sequences for consistent input size
    max_sequence_length = max(len(seq) for seq in sequences)
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

    # Create a model for word embeddings
    word_embedding_model = Sequential()
    word_embedding_model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_length))
    word_embedding_model.compile(loss='categorical_crossentropy', optimizer='adam')
    word_embeddings = word_embedding_model.predict(padded_sequences)

    # Create a model for document embeddings
    document_embedding_model = Sequential()
    document_embedding_model.add(LSTM(100))
    document_embedding_model.compile(loss='categorical_crossentropy', optimizer='adam')
    document_embeddings = document_embedding_model.predict(word_embeddings)

    # Add the embeddings to the DataFrame
    a['word_embeddings'] = list(word_embeddings)
    a['document_embeddings'] = list(document_embeddings)

# Process the ingredients column in the DataFrame
# Display the updated DataFrame

186/186 [==============================] - 1s 4ms/step


In [27]:
def get_recommendations(N, scores):
    """
    Rank scores and output a pandas data frame containing all the details of the top N recipes.
    :param scores: list of cosine similarities
    """
    # load in recipe dataset
    df_recipes = df.copy()
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations
    recommendation = pd.DataFrame(columns=["TranslatedRecipeName", "ingredients", "URL","score","cuisine"])
    count = 0
    for i in top:
        recommendation.at[count, "TranslatedRecipeName"] = title_parser(df_recipes["TranslatedRecipeName"][i])
        recommendation.at[count, "ingredients"] = ingredient_parser_final(
            df_recipes["ingredients"][i]
        )
        recommendation.at[count, "URL"] = df_recipes["URL"][i]
        recommendation.at[count, "cuisine"] = df_recipes["Cuisine"][i]
        recommendation.at[count, "score"] = f"{scores[i]}"
        count += 1
    return recommendation

def get_recs(ingredients, N=5, mean=False):
    """
    Get the top N recipe recomendations.
    :param ingredients: comma seperated string listing ingredients
    :param N: number of recommendations
    :param mean: False if using tfidf weighted embeddings, True if using simple mean
    

    data=df.copy()
    # parse ingredients
    data["parsed"] = data.ingredients.apply(ingredient_parser)
    # create corpus
    corpus = get_and_sort_corpus(data)

    if mean:
        # get average embdeddings for each document
        mean_vec_tr = MeanEmbeddingVectorizer(model)
        doc_vec = mean_vec_tr.transform(corpus)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
        assert len(doc_vec) == len(corpus)
    else:
        # use TF-IDF as weights for each word embedding
        tfidf_vec_tr = TfidfEmbeddingVectorizer(model)
        tfidf_vec_tr.fit(corpus)
        doc_vec = tfidf_vec_tr.transform(corpus)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
        assert len(doc_vec) == len(corpus)
    """

    # create embeddings for input text
    input = ingredients
    # create tokens with elements
    input = input.split(",")
    # parse ingredient list
    input = ingredient_parser(input)
    input = process_ingredients_input(input)
    # get embeddings for ingredient doc
    

    # get cosine similarity between input embedding and all the document embeddings
    a_document_embeddings = a['document_embeddings'].values[0]

# Calculate cosine similarities using a list comprehension
    cos_sim = [cosine_similarity([a_document_embeddings], x.reshape(1, -1))[0][0] for x in df['document_embeddings']]


# Assuming 'a' contains the document embeddings you want to compare to 'df'
    #a_document_embeddings = a['document_embeddings'].values[0]

# Calculate cosine similarities using a list comprehension
# 'cos_sim' will now contain the cosine similarities
    scores = list(cos_sim)
    # Filter top N recommendations
    recommendations = get_recommendations(N, scores)
    return recommendations

In [32]:
if __name__ == "__main__":
    # test 
    input = "chicken, egg, tomato, onion"
    rec = get_recs(input)
    print(rec)

1/1 [==============================] - 0s 240ms/step
                                TranslatedRecipeName  \
0                  Iyengar Style Thavala Adai Recipe   
1  Thandai Phirni Recipe - A Delicious Indian Ric...   
2                         Herbed Dinner Rolls Recipe   
3                           Bengali Steam Doi Recipe   
4                         Mushroom Fried Rice Recipe   

                                         ingredients  \
0  salt,arhar dal ,rice ,mustard seeds,methi powd...   
1  thandai powder,cashew nuts,rice,pistachios,saf...   
2  salt,eggs fork,milk,water,dry yeast,herbs,viva...   
3  hung curd ,pistachios,milk,raisins,strawberrie...   
4  basmati rice,spring onion greens,ginger,salt,g...   

                                                 URL                score  \
0  https://www.archanaskitchen.com/iyengar-style-...  0.24577851593494415   
1  https://www.archanaskitchen.com/thandai-phirni...  0.24188458919525146   
2  https://www.archanaskitchen.com/herbed-

In [33]:
test = pd.read_csv("/Users/ved/Desktop/Capstone/accuracytest_-_Sheet1_1.csv")
actual_recipes = list(test["TranslatedRecipeName"])

In [ ]:
precision_at_K =

In [41]:
K = 10 # mention it as actual no. of recipies
# Calculate P@K
correct_recommendations=[]
for recipe in actual_recipes:
    for i in rec['TranslatedRecipeName'][:K]:
        if recipe == i:
            correct_recommendations.append(True)
#correct_recommendations = [recipe in rec['TranslatedRecipeName'][:K] for recipe in actual_recipes]
precision_at_K = sum(correct_recommendations) / K

print(f"Precision at {K} using LSTM: {precision_at_K}")

Precision at 10 using LSTM: 0.0
